In [1]:
from bs4 import BeautifulSoup
import time
import os
import urllib.request
from urllib.error import URLError, HTTPError

from selenium import webdriver
import re
import pandas as pd

In [ ]:
path = "C:/chromedriver.exe" # chromedriver.exe 다운로드 필수
driver = webdriver.Chrome(path)

In [ ]:
url1='https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptList.do?pageIndex=1&currentPage='
url2='&sortField=exprRnkn&setZzimBusiId=&bizId=&plcySel=&chargerOrgCdSel=&sortField2=exprRnkn&higdBlowAccrYnSel=&hiscEnrlYnSel=&hiscGrdnPrerYnSel=&higdAccrYnSel=&univEnrlYnSel=&univGrdnAccrYnSel=&drAccrYnSel=&majrSglSersYnSel=&majrSocySersYnSel=&majrBsflSersYnSel=&majrNtssSersYnSel=&majrEngrSersYnSel=&majrEnspSersYnSel=&splzRlmSmpzEmpmYnSel=&splzRlmFmleYnSel=&splzRlmLigYnSel=&splzRlmDspnYnSel=&splzRlmSdrYnSel=&splzRlmFrmlYnSel=&empnHffcYnSel=&empyBsmgYnSel=&empmUnpnYnSel=&scroll=1&btnSearchTxt=&srchAge=&higdBlowAccrYn=on&hiscEnrlYn=on&hiscGrdnPrerYn=on&higdAccrYn=on&univEnrlYn=on&univGrdnAccrYn=on&drAccrYn=on&majrSglSersYn=on&majrSocySersYn=on&majrBsflSersYn=on&majrNtssSersYn=on&majrEngrSersYn=on&majrEnspSersYn=on&splzRlmSmpzEmpmYn=on&splzRlmFmleYn=on&splzRlmLigYn=on&splzRlmDspnYn=on&splzRlmSdrYn=on&splzRlmFrmlYn=on&empnHffcYn=on&empyBsmgYn=on&empmUnpnYn=on'
page_num=1

In [ ]:
regex_digit = re.compile("\'\d+\'")

In [ ]:
regex_digit2 = re.compile("\d+")

while로 교체필요

In [28]:
def id_crawl():
    
    res_list=[]
    
    for page_num in range(1,22):
        driver.get(url1 + str(page_num)+url2)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        res_ids = regex_digit.findall(str(soup.select('#jynEmpSptList1')))
        res_list.extend(res_ids)
        
    return res_list

In [29]:
id_list = id_crawl()

In [30]:
def del_up_dot(ids_list):
    res = []
    for i in ids_list:
        x=regex_digit2.findall(i)
        res.extend(x)
        
    return res

In [31]:
id_list2 = del_up_dot(id_list)

In [35]:
url_id_base = "https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId="

In [36]:
col_list = ['id','title','hash tag','설명','연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']

In [37]:
pattern = re.compile(r'[^\t\n]+')

In [38]:
info_table=pd.DataFrame(columns=col_list)

In [44]:
def chung_crawl(ids_list):
    num_df = 0
    col_list = ['id','title','hash tag','설명','연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']
    index_list = ['연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']
    info_table=pd.DataFrame(columns=col_list)
    for i in ids_list:
        driver.get(url_id_base+str(i))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
    
        sample_list=[]
        exp=[]
        exp.append(i)
        idx=soup.select('h3.doc_tit02')[0].text
        string1 = pattern.findall(str(idx))
        exp.append(string1[0])
        if len(string1)>=3:
            exp.append(string1[2])
        else:
            exp.append('-')
    
    
        idx=soup.select('h4.bullet-arrow1')[0].text
        string2=pattern.findall(str(idx))
        if len(string2)!=0:
            exp.append(string2[0])
        else:
            exp.append('-')
  
    
        idx=soup.select('ul.common_table02')[0].text
        string=pattern.findall(str(idx))

        remove_space = list(filter(lambda x: x!=' ', string))
    
        for kda in index_list:
            num=remove_space.index(kda)
            exp.append(remove_space[num+1])
    

        info_table.loc[num_df]=exp
        num_df = num_df+1
    return info_table

In [45]:
df = chung_crawl(id_list2)

In [47]:
df['id'] = df['id'].apply(lambda x: url_id_base+str(x))

In [51]:
df.loc[1]['id']

'https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId=201808230013'

In [ ]:
pattern_